# Create video of the images in the folder

### Remove old variables (just in case)

In [1]:
# Remove old variables
#%reset

### Import libraries

In [2]:
import cv2
import os
import numpy as np

### Set paths and read images

In [3]:
# Set paths and video output name
image_folder = 'data/Varrio_Scanner3'
video_name = 'video_Varrio3.mp4'

# Set video frame size (scale)
scale=0.5

# Set font size
font_size=3

# Number of pictures in second
fps=3

### Get image paths and set video settings

In [4]:
# Get image names
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]

# Sort image names
images = sorted(images)

# Set video frame size
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
height=int(scale*height)
width=int(scale*width)
dim_vid=(width, height*2)
dim_pic=(width, height)

# Set video settings
video = cv2.VideoWriter(filename=video_name, fourcc=cv2.VideoWriter_fourcc(*'XVID'), fps=fps, frameSize=dim_vid)

# Set text font
font = cv2.FONT_HERSHEY_SIMPLEX

### Create only root images

In [5]:
# Detect roots from image
def get_roots(img_color):
    
    # Convert to gray scale
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

    # Remove noise with gaussian blur (3, 3 works)
    edges = cv2.GaussianBlur(gray,(3,3),0)

    # Edges to binary (black and white) (180 and 250)
    ret,edges = cv2.threshold(edges,180,256,cv2.THRESH_BINARY)

    # Remove isolated pixels
    edges = cv2.GaussianBlur(edges,(5,5),0)
    edges = cv2.threshold(edges, 150 , 250, cv2.THRESH_BINARY)
    edges = edges[1]

    # Remove isolated pixels
    edges = cv2.erode(edges, None, iterations=1)
    edges = cv2.dilate(edges, None, iterations=1)

    # Return results
    return edges

### Convert pics to video and save output

In [6]:
# Initialize roots locations
roots=np.zeros((height,width), np.uint8)

# Make video of images
for image in images:
    
    # Read image
    im = cv2.imread(os.path.join(image_folder, image))
    
    # Resize image
    im = cv2.resize(im, dim_pic,interpolation = cv2.INTER_AREA)
    
    # Get new root location pixels
    new_roots=get_roots(im)
    
    # Add new root locations pixels to previous root locations pixels
    roots = new_roots+roots
    
    # Mask root locations to image (get colors to root pixels)
    im_masked = cv2.bitwise_or(im, im, mask=roots)
    
    # Concat original image and roots image vertically
    im = np.concatenate((im, im_masked), axis=0)
    
    # Get image date from the image name
    # Date is in different location for Hyde and Varrio
    #date = image[20:30] # For Hyde Scanners
    date = image[14:24] # For Varrio Scanners
    
    # Add date to upper left corner
    cv2.putText(img=im, text=date, org=(font_size*10,font_size*30), fontFace=font, fontScale=font_size, 
                color=(255, 255, 255), thickness=font_size, lineType=cv2.LINE_AA)
    
    # Add frame to video
    video.write(im)
     
# Close windows
cv2.destroyAllWindows() 

# Save output video
video.release()